#Deliverable-2
Student - Aditya Bhavsar



In [ ]:
#Required installation of the following library
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 1.9 MB/s eta 0:00:00


In [ ]:
!pip install readability

  Preparing metadata (setup.py) ... done
  Created wheel for readability: filename=readability-0.3.2-py3-none-any.whl size=36384 sha256=c8d141a08c943eef470e0af2043a70e5a9705364f7c04f8caf8b7bfd7304acdb
  Stored in directory: /root/.cache/pip/wheels/6a/a8/01/0b6587e224d9731dae317fdad11b081f0e8b7be7d8367fc6eb
Successfully built readability


In [ ]:
!pip install textstat

In [ ]:
import os
os.environ["HF_TOKEN"] = "Your Hf Token"

In [ ]:
from huggingface_hub import login
login(token=os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
import requests
import tldextract
from datetime import datetime
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import time
from textstat import flesch_reading_ease
import os

# Setup API tokens
# Replace with yourAPI Key
os.environ["HF_TOKEN"] = "Your hftoken"
SERP_API_KEY = "Your serp api key"

# === 1st metric - Domain Trust ===
def get_domain_trust(url):
    domain = tldextract.extract(url).registered_domain
    try:
        tranco_response = requests.get("https://tranco-list.eu/top-1m.csv").text
        if domain in tranco_response:
            tranco_rank = tranco_response.split(domain)[0].strip().split("\n")[-1]
            tranco_score = max(100 - (int(tranco_rank) / 10000), 50)
        else:
            tranco_score = 40
    except:
        tranco_score = 40

    try:
        whois_response = requests.get(f"https://api.ip2whois.com/v2?key=demo&domain={domain}").json()
        creation_date = whois_response.get("created_date", "2000-01-01")
        domain_age = (datetime.now() - datetime.strptime(creation_date, "%Y-%m-%d")).days // 365
        age_score = min(domain_age * 5, 100)
    except:
        age_score = 50

    try:
        search = requests.get(f"https://serpapi.com/search.json?q={domain}&api_key={SERP_API_KEY}").json()
        backlink_count = len(search.get("organic_results", []))
        backlink_score = min(backlink_count * 10, 100)
    except:
        backlink_score = 50

    domain_trust = (0.4 * tranco_score) + (0.3 * age_score) + (0.3 * backlink_score)
    return round(domain_trust, 2)

# === 2nd metric - AI Based Fact-Check Score ===
def get_fact_check_score(text):
    try:
        params = {
            "q": f"fact check {text}",
            "engine": "google",
            "api_key": SERP_API_KEY
        }
        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()
        trusted_sources = ["snopes.com", "politifact.com", "factcheck.org", "bbc.com", "reuters.com"]
        source_mentions = sum(1 for result in data.get("organic_results", [])
                              if any(domain in result.get("link", "") for domain in trusted_sources))
        fact_check_score = min(source_mentions * 20, 100)
    except Exception:
        fact_check_score = 50

    try:
        wiki_response = requests.get(
            f"https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch={text}"
        )
        wiki_data = wiki_response.json()
        wiki_matches = len(wiki_data.get("query", {}).get("search", []))
        fact_check_score += min(wiki_matches * 10, 30)
    except Exception:
        pass

    try:
        model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        trusted_texts = " ".join([
            result["title"] + result["snippet"]
            for result in data.get("organic_results", [])
            if "title" in result and "snippet" in result
        ])
        similarity_score = util.pytorch_cos_sim(
            model.encode(text), model.encode(trusted_texts)
        ).item() * 100
        fact_check_score += min(similarity_score / 2, 30)
    except Exception:
        pass

    return max(0, min(fact_check_score, 100))

# === 3rd metric - Bias Score ===
def get_bias_score(text, domain):
    sentiment_pipeline = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
    sentiment_result = sentiment_pipeline(text[:512])[0]
    sentiment_bias = {
        "1 star": 30,
        "2 stars": 50,
        "3 stars": 70,
        "4 stars": 80,
        "5 stars": 100
    }
    bias_score = sentiment_bias.get(sentiment_result["label"], 50)
    try:
        bias_response = requests.get(f"https://api.allsides.com/bias/{domain}").json()
        media_bias = bias_response.get("bias", "center")
        bias_adjustment = {
            "left": -20,
            "lean left": -10,
            "center": 0,
            "lean right": 10,
            "right": 20
        }.get(media_bias.lower(), 0)
        bias_score = max(0, min(bias_score + bias_adjustment, 100))
    except:
        pass
    return round(bias_score, 2)

# === 4th metric - Content Relevance Score ===
def compute_content_relevance(user_query, page_text):
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    similarity_score = util.pytorch_cos_sim(
        model.encode(user_query), model.encode(page_text)
    ).item() * 100
    return round(similarity_score, 2)

# === 5th metric - Citation Score ===
def check_google_scholar(url):
    try:
        backlink_params = {
            "q": f"link:{url}",
            "engine": "google",
            "api_key": SERP_API_KEY
        }
        backlink_response = requests.get("https://serpapi.com/search", params=backlink_params).json()
        backlink_count = len(backlink_response.get("organic_results", []))

        academic_params = {
            "q": f"\"{url}\" filetype:pdf OR site:researchgate.net OR site:arxiv.org OR site:semanticscholar.org",
            "engine": "google",
            "api_key": SERP_API_KEY
        }
        academic_response = requests.get("https://serpapi.com/search", params=academic_params).json()
        academic_count = len(academic_response.get("organic_results", []))

        citation_score = min((backlink_count * 5) + (academic_count * 15), 100)
        return round(citation_score, 2)
    except Exception as e:
        print(f"Error fetching citation data: {e}")
        return 0

# === 6th metric - Page Load Speed Score ===
def get_page_load_speed(url):
    start_time = time.time()
    try:
        response = requests.get(url, timeout=10)
        load_time = time.time() - start_time
        return max(0, min(100 - (load_time * 10), 100))
    except:
        return 50

# === 7th metric - Plagiarism Check Score ===
def check_plagiarism(text):
    try:
        search_query = f'"{text[:100]}"'
        response = requests.get(f"https://serpapi.com/search?q={search_query}&api_key={SERP_API_KEY}")
        duplicate_results = len(response.json().get("organic_results", []))
        return max(0, 100 - (duplicate_results * 20))
    except:
        return 50

# === 8th metric - Readability Score ===
def get_readability_score(text):
    try:
        score = flesch_reading_ease(text)
        return max(0, min(score, 100))
    except:
        return 50

# === 9th metric - SSL Security Score ===
def check_ssl_security(url):
    try:
        domain = tldextract.extract(url).registered_domain
        response = requests.get(f"https://{domain}", timeout=5)
        return 100 if response.url.startswith("https") else 0
    except:
        return 0

# === 10th metric - Language Complexity & Coherence Score ===
def check_language_complexity(text):
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    try:
        coherence_score = util.pytorch_cos_sim(
            model.encode(text[:500]),
            model.encode("High-quality journalistic content.")
        ).item() * 100
        return round(coherence_score, 2)
    except:
        return 50

# === 11th metric - User Engagement Score ===
def get_user_engagement(url):
    try:
        response = requests.get(f"https://serpapi.com/search?q=site:{url}&api_key={SERP_API_KEY}")
        social_mentions = len(response.json().get("organic_results", []))
        return min(social_mentions * 10, 100)
    except:
        return 50

# === Helper Function: Convert Score to Star Rating ===
def get_star_rating(score: float) -> tuple:
    stars = max(1, min(5, round(score / 20)))
    return stars, "⭐" * stars

# === Updated Helper Function: Generate Explanation using All Detailed Metrics ===
def generate_explanation(metrics: dict, final_score: float) -> str:
    """
    Generates a human-readable explanation based on all detailed metrics.
    """
    reasons = []
    if metrics.get("Domain Trust", 0) < 50:
        reasons.append("The source has low domain authority.")
    if metrics.get("Content Relevance", 0) < 50:
        reasons.append("The content is not highly relevant to your query.")
    if metrics.get("Fact-Check Score", 0) < 50:
        reasons.append("Limited fact-checking verification found.")
    if metrics.get("Bias Score", 0) < 50:
        reasons.append("Potential bias detected in the content.")
    if metrics.get("Citation Score", 0) < 30:
        reasons.append("Few citations found for this content.")
    if metrics.get("Page Load Speed Score", 0) < 50:
        reasons.append("The page load speed is slow.")
    if metrics.get("Plagiarism Score", 0) < 50:
        reasons.append("High similarity to other sources detected.")
    if metrics.get("Readability Score", 0) < 50:
        reasons.append("The content is difficult to read.")
    if metrics.get("SSL Security Score", 0) < 50:
        reasons.append("The website may not be secure.")
    if metrics.get("Language Coherence Score", 0) < 50:
        reasons.append("The language coherence is below expectations.")
    if metrics.get("User Engagement Score", 0) < 50:
        reasons.append("Low user engagement detected.")

    if not reasons:
        reasons.append("This source is highly credible and relevant.")

    explanation = " ".join(reasons) + f" Overall credibility score: {round(final_score, 2)}."
    return explanation

# === Final Validity Evaluation using All Metrics (Detailed Score) ===
def rate_url_validity(user_query: str, url: str) -> dict:
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        page_text = " ".join([p.text for p in soup.find_all("p")])
    except Exception:
        return {"error": "Failed to fetch content."}

    domain = tldextract.extract(url).registered_domain

    # Compute each metric
    domain_trust       = get_domain_trust(url)
    content_relevance  = compute_content_relevance(user_query, page_text)
    fact_check_score   = get_fact_check_score(page_text)
    bias_score         = get_bias_score(page_text, domain)
    citation_score     = check_google_scholar(url)
    page_load_speed    = get_page_load_speed(url)
    plagiarism_score   = check_plagiarism(page_text)
    readability_score  = get_readability_score(page_text)
    ssl_security       = check_ssl_security(url)
    language_coherence = check_language_complexity(page_text)
    user_engagement    = get_user_engagement(url)

    # Detailed final score calculation using all metrics and weights
    final_score_detailed = (
        (0.10 * domain_trust) +
        (0.50 * content_relevance) +
        (0.05 * fact_check_score) +
        (0.05 * bias_score) +
        (0.10 * citation_score) +
        (0.05 * page_load_speed) +
        (0.05 * plagiarism_score) +
        (0.025 * readability_score) +
        (0.03 * ssl_security) +
        (0.02 * language_coherence) +
        (0.025 * user_engagement)
    )

    # Gather metrics into a dictionary
    metrics = {
        "Domain Trust": domain_trust,
        "Content Relevance": content_relevance,
        "Fact-Check Score": fact_check_score,
        "Bias Score": bias_score,
        "Citation Score": citation_score,
        "Page Load Speed Score": page_load_speed,
        "Plagiarism Score": plagiarism_score,
        "Readability Score": readability_score,
        "SSL Security Score": ssl_security,
        "Language Coherence Score": language_coherence,
        "User Engagement Score": user_engagement
    }

    # Use detailed score for star rating and explanation
    stars, star_icon = get_star_rating(final_score_detailed)
    explanation = generate_explanation(metrics, final_score_detailed)

    return {
        #"raw_score": metrics,
        "final_score_detailed": round(final_score_detailed, 2),
        "stars": {
            "score": stars,
            "icon": star_icon
        },
        "explanation": explanation
    }

# === Example Usage ===
if __name__ == "__main__":
    user_query = "Nvidias rtx 5070 new gpu is it really good ?"
    url = "https://www.fool.com/investing/2025/02/09/is-nvidia-still-a-millionaire-maker-stock/"
    result = rate_url_validity(user_query, url)
    print(result)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


{'final_score_detailed': 43.04, 'stars': {'score': 2, 'icon': '⭐⭐'}, 'explanation': 'The content is not highly relevant to your query. Limited fact-checking verification found. High similarity to other sources detected. The content is difficult to read. The language coherence is below expectations. Low user engagement detected. Overall credibility score: 43.04.'}


In [ ]:
if __name__ == "__main__":
    user_query = "Nvidia new RTX 5070 is it really good ?"
    url = "https://www.tomshardware.com/pc-components/gpus/nvidias-geforce-rtx-5070-at-usd549-how-does-it-stack-up-to-the-previous-generation-rtx-4070"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 62.91, 'stars': {'score': 3, 'icon': '⭐⭐⭐'}, 'explanation': 'Limited fact-checking verification found. The language coherence is below expectations. Low user engagement detected. Overall credibility score: 62.91.'}


In [ ]:
if __name__ == "__main__":
    # Replace with your query and URL to test
    user_query = "kingdome come deliverance 2 got good rating compare to one give me a proper review about this game"
    url = "https://www.rpgsite.net/review/16825-kingdom-come-deliverance-ii-review"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 59.01, 'stars': {'score': 3, 'icon': '⭐⭐⭐'}, 'explanation': 'High similarity to other sources detected. The content is difficult to read. The language coherence is below expectations. Low user engagement detected. Overall credibility score: 59.01.'}


In [ ]:
if __name__ == "__main__":
    # Replace with your query and URL to test
    user_query = "kingdome come deliverance 2 got good rating compare to one give me a proper review about this game"
    url = "https://www.kingdomcomehome.shop/"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 40.75, 'stars': {'score': 2, 'icon': '⭐⭐'}, 'explanation': 'The content is not highly relevant to your query. Limited fact-checking verification found. High similarity to other sources detected. The content is difficult to read. The language coherence is below expectations. Overall credibility score: 40.75.'}


In [ ]:
if __name__ == "__main__":
    user_query = "delta airlines planes crashed in toronto airport give me some relevant news regarding it"
    url = "https://www.usatoday.com/story/travel/news/2025/02/17/delta-regional-flight-crashes-toronto-airport/78983808007/"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 56.8, 'stars': {'score': 3, 'icon': '⭐⭐⭐'}, 'explanation': 'Limited fact-checking verification found. Potential bias detected in the content. High similarity to other sources detected. The content is difficult to read. The language coherence is below expectations. Low user engagement detected. Overall credibility score: 56.8.'}


In [ ]:
if __name__ == "__main__":
    user_query = "deep learning identifying sleep stages or patterns"
    url = "https://developer.nvidia.com/blog/deep-learning-can-effectively-identify-sleep-stages/"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 65.64, 'stars': {'score': 3, 'icon': '⭐⭐⭐'}, 'explanation': 'Limited fact-checking verification found. The content is difficult to read. The language coherence is below expectations. Low user engagement detected. Overall credibility score: 65.64.'}


In [ ]:
if __name__ == "__main__":
    user_query = "was moon landing by NASA fake!"
    url = "https://www.bbc.com/news"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 37.45, 'stars': {'score': 2, 'icon': '⭐⭐'}, 'explanation': 'The content is not highly relevant to your query. Limited fact-checking verification found. High similarity to other sources detected. The language coherence is below expectations. Overall credibility score: 37.45.'}


In [ ]:
if __name__ == "__main__":
    user_query = "does whey protein deteriorate kidney function"
    url = "https://pubmed.ncbi.nlm.nih.gov/32702243/"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 64.31, 'stars': {'score': 3, 'icon': '⭐⭐⭐'}, 'explanation': 'Limited fact-checking verification found. The content is difficult to read. The language coherence is below expectations. Low user engagement detected. Overall credibility score: 64.31.'}


In [ ]:
if __name__ == "__main__":
    user_query = "will amd FSR get better than nvidia dlss 4"
    url = "https://www.techradar.com/computing/gpu/nvidias-dlss-4-is-amazing-heres-what-amds-fsr-4-needs-to-do-to-take-it-on"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 71.16, 'stars': {'score': 4, 'icon': '⭐⭐⭐⭐'}, 'explanation': 'Limited fact-checking verification found. The content is difficult to read. The language coherence is below expectations. Low user engagement detected. Overall credibility score: 71.16.'}


In [ ]:
if __name__ == "__main__":
    user_query = "will amd FSR get better than nvidia dlss 4"
    url = "https://steamcommunity.com/discussions/forum/11/3543798390532636155/"
    result = rate_url_validity(user_query, url)
    print(result)

Device set to use cpu


{'final_score_detailed': 31.51, 'stars': {'score': 2, 'icon': '⭐⭐'}, 'explanation': 'The content is not highly relevant to your query. Few citations found for this content. The language coherence is below expectations. Overall credibility score: 31.51.'}
